In [68]:
import numpy as np
import pandas as pd

In [69]:
df = pd.read_csv("../../data/jx_duty.csv")
df.head(5)

,Unnamed: 0,X,shixin_id,duty
0,4,4,18095,被告李华于判决生效之日起三日内给付原告郭万清购车款人民币元案件受理费元 邮寄费元 合计元 由...
1,5,5,18196,一被告任晓平偿还原告罗秀花欠款元二本案诉讼费元 由被告任晓平自愿承担
2,6,6,18542,被告寇先强于本判决生效之日起十日内赔偿给原告共计元
3,7,7,21388,判决如下 一被告陈雪明向原告农四师农业生产资料公司六十四团分公司支付农业生产资料款元 利息元...
4,8,8,21456,被告苏宪礼自愿赔偿原告王世霜各项经济损失元整此款被告苏宪礼于年月日前支付元 于年月日前支付元...


In [70]:
df.shape

(50284, 4)

In [71]:
df = df.sample(20000)

In [72]:
import jieba

In [73]:
# Need to be length greater than 10;

def length_bigger_than_10(text):
    if len(text)>=10:
        return True
    else:
        return False


#新建length列
df['length'] = df.agg({'duty': length_bigger_than_10})
#根据length列来保留长度大于10的文本，并将其放入新的dataframe中
df = df[df['length']==True]

In [74]:
import nltk
import jieba

In [75]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leijunjie/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [76]:
#停止词
from nltk.corpus import stopwords
stopwords = stopwords.words("chinese")



def clean_text(text):
    wordlist = jieba.lcut(text) 
    #去除停用词和长度小于2的词语
    wordlist = [w for w in wordlist if w not in stopwords and len(w)>2]
    #将中文数据组织成类似西方于洋那样，词语之间以空格间隔
    document =  " ".join(wordlist)
    return document


#使用dataframe.agg对content列实行clean_text方法
df['duty'] = df.agg({'duty': clean_text})
corpus = df['duty']
corpus.head()

22971                                      陈静于 陈静于 杨金翠 杨金翠
43261    赵思刚 十五日 中国农业银行 博尔塔拉 原被告 刘玉华吐尔松 买买提 吐然克孜卡 赵思刚 连...
36916                                         法律效力 一次性 拜城县
17645               赵志虎于 二十日 赵小明 赵志虎于 二十日 赵小明 赵志虎于 二十日 代理费
15900                                 三十日 交通费 鉴定费 交通费 诉讼费用
Name: duty, dtype: object

In [77]:
## jieba 分词包

def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [78]:
df["duty_cutted"] = df.duty.apply(chinese_word_cut)

In [79]:
df.duty_cutted.head()

22971                                陈静于   陈静于   杨金翠   杨金翠
43261    赵思刚   十五日   中国农业银行   博尔塔拉   原被告   刘玉华吐尔松   买买提...
36916                                     法律效力   一次性   拜城县
17645    赵志虎于   二十日   赵小明   赵志虎于   二十日   赵小明   赵志虎于   二...
15900                         三十日   交通费   鉴定费   交通费   诉讼费用
Name: duty_cutted, dtype: object

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [81]:
stpwrdpath = "../../data/stopwords.txt"
stpwrd_dic = open(stpwrdpath, 'rb')
stpwrd_content = stpwrd_dic.read().decode('utf-8')#将停用词表转换为list
stpwrdlst = stpwrd_content.splitlines()
stpwrd_dic.close()

In [82]:
n_features = 1000

In [83]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                #stop_words='english',
                                stop_words=stpwrdlst,
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.duty_cutted)

In [84]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                #stop_words='english',
                                stop_words=stpwrdlst,
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(corpus)

In [85]:
from sklearn.decomposition import LatentDirichletAllocation

In [86]:
lda = LatentDirichletAllocation(n_components = 30,max_iter=50,
                                learning_method='online',
                                learning_offset=50,
                                random_state=0)

In [87]:
lda.fit(tf)

LatentDirichletAllocation(learning_method='online', learning_offset=50,
                          max_iter=50, n_components=30, random_state=0)

In [88]:
import pyLDAvis 
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
4      0.031734  0.336694       1        1  9.300264
26     0.324815 -0.012059       2        1  7.963636
14     0.356092 -0.101404       3        1  7.093629
21    -0.029708 -0.016296       4        1  5.013490
0     -0.079963 -0.022152       5        1  4.865719
28    -0.032712 -0.026263       6        1  4.743155
23     0.000913  0.129845       7        1  4.433619
5     -0.056786 -0.034664       8        1  4.080982
25    -0.030524  0.110972       9        1  3.913913
7     -0.033156  0.003129      10        1  3.747929
18    -0.037539 -0.030151      11        1  3.704152
1     -0.039641 -0.034084      12        1  3.369757
11    -0.034081  0.116670      13        1  3.142954
6      0.079732  0.259906      14        1  2.809043
22    -0.037844 -0.031250      15        1  2.537356
12    -0.085278 -0.045779      16        1  2.531307
20    -0.038822 -0.021491      17        1  2.491673
16    -0.043010 -0.039438      18        1  2.468496
19     0.171585 -0.141109      19        1  2.442944
10    -0.039906 -0.035017      20        1  2.273950
9     -0.038943 -0.033544      21        1  2.225605
2      0.086885  0.043086      22        1  2.071618
13    -0.055854 -0.053923      23        1  1.992843
15    -0.049419 -0.025942      24        1  1.903461
29    -0.046756 -0.046846      25        1  1.870242
27    -0.045387 -0.044815      26        1  1.815261
8     -0.048022 -0.049738      27        1  1.423409
3     -0.052229 -0.053050      28        1  1.380051
24    -0.048670 -0.051288      29        1  1.237101
17    -0.047506 -0.049999      30        1  1.152442, topic_info=      Term         Freq        Total Category  logprob  loglift
5      一次性  2454.000000  2454.000000  Default   30.000  30.0000
284    房地产  1565.000000  1565.000000  Default   29.000  29.0000
150    十五日  1171.000000  1171.000000  Default   28.000  28.0000
509   连带责任  1134.000000  1134.000000  Default   27.000  27.0000
30    乌鲁木齐   880.000000   880.000000  Default   26.000  26.0000
..     ...          ...          ...      ...      ...      ...
361    沙湾县     0.052155   155.170244  Topic30   -9.474  -3.5348
404   社会保险     0.052155    33.317100  Topic30   -9.474  -1.9963
6    一百四十四     0.052155    37.183248  Topic30   -9.474  -2.1061
352    案一审     0.052155    90.873887  Topic30   -9.474  -2.9997
461    补偿款     0.052155    55.894500  Topic30   -9.474  -2.5137

[1228 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         2  0.948453   一万元
1        27  0.970120  一三四团
2        30  0.993838   一个月
3        24  0.964921   一二三
4         1  0.971765  一审判决
...     ...       ...   ...
587      16  0.978323   魏德平
588      25  0.973615   魏昌会
589      28  0.968230  麦盖提县
590       1  0.975108   黄洪松
591      20  0.956696   齐新德

[626 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 27, 15, 22, 1, 29, 24, 6, 26, 8, 19, 2, 12, 7, 23, 13, 21, 17, 20, 11, 10, 3, 14, 16, 30, 28, 9, 4, 25, 18])

In [89]:
p = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [90]:
pyLDAvis.save_html(p, 'lda_10_xj.html')

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
